In [21]:
import nltk
from nltk.corpus import semcor
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Download dữ liệu cần thiết từ NLTK (SemCor và WordNet)
nltk.download('semcor')
nltk.download('wordnet')

[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
def extract_instances(target_lemma):
    """
    Trích xuất các instance từ SemCor cho từ mục tiêu (target_lemma).
    Mỗi instance gồm:
      - Đặc trưng được trích xuất từ ngữ cảnh gồm 2 từ trước và 2 từ sau,
        bao gồm cả POS tags và 2 bigram (các cặp từ liên tiếp).
      - Nhãn: tên synset của từ (là nhãn của nghĩa được gán).
    
    Cụ thể, các đặc trưng được xây dựng theo cách:
      * Các từ và POS ở vị trí i-2, i-1, i+1, i+2.
      * Các bigram: (i-2, i-1) và (i+1, i+2).
    Các từ gần vị trí cần phân giải (i-1, i+1) được gán trọng số cao hơn.
    """
    instances = []
    # Duyệt qua từng câu có gán nhãn trong SemCor
    for sent in semcor.tagged_sents(tag='both'):
        # Đảm bảo có đủ ngữ cảnh: bắt đầu từ vị trí thứ 2 đến len(sent)-2
        for i in range(2, len(sent) - 2):
            word_tree = sent[i]
            if isinstance(word_tree, nltk.Tree):
                # Lấy nhãn nghĩa của từ; lưu ý: cách gán nhãn phụ thuộc vào cấu trúc của SemCor
                sense_tag = word_tree.label()
                # Kiểm tra xem sense_tag có hỗ trợ phương thức synset() không
                if hasattr(sense_tag, 'synset'):
                    synset = sense_tag.synset()
                    # Lấy lemma từ tên synset (ví dụ: "bank.n.01" -> "bank")
                    lemma = synset.name().split('.')[0]
                    if lemma == target_lemma:
                        # Trích xuất từ và POS từ ngữ cảnh:
                        # Lấy 2 từ trước
                        wi_m2 = ' '.join(sent[i-2].leaves())
                        pos_m2 = sent[i-2].label()
                        wi_m1 = ' '.join(sent[i-1].leaves())
                        pos_m1 = sent[i-1].label()
                        # Lấy 2 từ sau
                        wi_p1 = ' '.join(sent[i+1].leaves())
                        pos_p1 = sent[i+1].label()
                        wi_p2 = ' '.join(sent[i+2].leaves())
                        pos_p2 = sent[i+2].label()
                        
                        # Tạo bigram từ ngữ cảnh bên trái và bên phải
                        bigram_left = f"{wi_m2}_{wi_m1}"
                        bigram_right = f"{wi_p1}_{wi_p2}"
                        
                        # Tạo dictionary đặc trưng với trọng số khác nhau:
                        features = {
                            f"Wi-2:{wi_m2}": 1,       # Từ cách 2 vị trí trước
                            f"POSi-2:{pos_m2}": 1,     # POS tương ứng
                            f"Wi-1:{wi_m1}": 2,        # Từ ngay trước (ưu tiên hơn)
                            f"POSi-1:{pos_m1}": 2,
                            f"Wi+1:{wi_p1}": 2,        # Từ ngay sau (ưu tiên hơn)
                            f"POSi+1:{pos_p1}": 2,
                            f"Wi+2:{wi_p2}": 1,        # Từ cách 2 vị trí sau
                            f"POSi+2:{pos_p2}": 1,
                            f"bigram_left:{bigram_left}": 1,   # Bigram bên trái
                            f"bigram_right:{bigram_right}": 1  # Bigram bên phải
                        }
                        # Lưu instance: (features, label) với label là tên synset
                        instances.append((features, synset.name()))
    return instances

In [48]:
def main():
    target_lemma = 'bar'  # Thay đổi từ mục tiêu nếu cần
    instances = extract_instances(target_lemma)
    
    if not instances:
        print(f"Không tìm thấy instance cho từ '{target_lemma}'")
        return
    
    # Tách đặc trưng và nhãn từ các instance
    feature_dicts, labels = zip(*instances)
    
    # Sử dụng DictVectorizer để chuyển đổi các dictionary đặc trưng thành ma trận số
    vectorizer = DictVectorizer(sparse=True)
    X = vectorizer.fit_transform(feature_dicts)
    
    # Áp dụng TF-IDF để cân bằng trọng số các đặc trưng
    transformer = TfidfTransformer()
    X_tfidf = transformer.fit_transform(X)
    
    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)
    
    # Huấn luyện mô hình kNN; sử dụng khoảng cách cosine thường phù hợp với dữ liệu TF-IDF
    knn = KNeighborsClassifier(n_neighbors=5, metric='cosine')
    knn.fit(X_train, y_train)
    
    # Dự đoán trên tập kiểm tra
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy cho từ '{target_lemma}': {accuracy:.2f}")

In [49]:
if __name__ == "__main__":
    main()

Accuracy cho từ 'bar': 0.75
